In [ ]:
import pandas as pd 
from matplotlib import pyplot as plt 
from sklearn.preprocessing import MinMaxScaler

import numpy as np 

%matplotlib inline

def Unique_Column(df):
    for column in df:
        print(f'{column} : {df[column].unique()}')

df = pd.read_csv("dataset.csv")

df.drop("customerID", axis = "columns", inplace = True)

pd.to_numeric(df.TotalCharges, errors="coerce")
df[pd.to_numeric(df.TotalCharges, errors="coerce").isnull()]

df1 = df[df.TotalCharges!=' ']
df1.loc[:, "TotalCharges"] = pd.to_numeric(df1["TotalCharges"])


df1 = df1.copy()
df1 = df1.replace({"No internet service": "No", "No phone service": "No"}).infer_objects(copy = False)

Y_N_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'Churn']

df1.loc[:, Y_N_cols] = df1.loc[:, Y_N_cols].replace({"Yes": 1, "No": 0}).infer_objects(copy=False).astype("Int64")


df2 = pd.get_dummies(data = df1,columns= ['InternetService', 'PaymentMethod', 'Contract'], dtype="Int64")
df2.sample(5)
df2.dtypes

scaler = MinMaxScaler()
Cols_Scale = ["tenure", "MonthlyCharges", "TotalCharges"]
df2[Cols_Scale] = scaler.fit_transform(df2[Cols_Scale])
df2.sample(5)


In [ ]:
from sklearn.model_selection import train_test_split

X = df2.drop("Churn", axis = "columns")
Y = df2["Churn"]

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)
X_train.shape

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

X_train_np = X_train.to_numpy().astype("float32") if hasattr(X_train, "to_numpy") else np.asarray(X_train, dtype="float32")
Y_train_np = Y_train.to_numpy().astype("float32") if hasattr(Y_train, "to_numpy") else np.asarray(Y_train, dtype="float32")

model = keras.Sequential([
    keras.layers.Dense(17, input_shape=(17,), activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(13, activation='relu'),
    keras.layers.Dense(11, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_np, Y_train_np, epochs=100)


In [ ]:
X_test_np = X_test.to_numpy().astype("float32") if hasattr(X_test, "to_numpy") else np.asarray(X_test, dtype="float32")
Y_test_np = Y_test.to_numpy().astype("float32") if hasattr(Y_test, "to_numpy") else np.asarray(Y_test, dtype="float32")

loss, acc = model.evaluate(X_test_np, Y_test_np, verbose=0)
print("Test loss:", loss)
print("Test accuracy:", acc)

yp = model.predict(X_test_np)

Y_preds = []
for i in yp:
    if i > 0.5:
        Y_preds.append(1)
    else:
        Y_preds.append(0)

from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(Y_test_np, Y_preds))

import seaborn as sn
cm = tf.math.confusion_matrix(labels = Y_test_np, predictions= Y_preds)

plt.figure(figsize = (10, 7))
sn.heatmap(cm, annot = True, fmt = 'd')
plt.xlabel("Predicted")
plt.ylabel("Truth")

In [ ]:
tenure_churn_leaves = df1[df1.Churn == 0].tenure
tenure_churn_Stays = df1[df1.Churn == 1].tenure


plt.hist([tenure_churn_leaves, tenure_churn_Stays], color = ['Blue', 'Red'],label = ["Leaves", "Stays"])
plt.xlabel("Total Time With Company(Tenure)")
plt.ylabel("No. of Customers")
plt.title("Visualization of relation between Tenure and Churn")
plt.legend()

In [ ]:
Support_Churn_Leaves = df1.loc[df1.Churn == 0, 'TechSupport']
Support_Churn_Stays  = df1.loc[df1.Churn == 1, 'TechSupport']

plt.hist([Support_Churn_Leaves, Support_Churn_Stays], color = ['Orange', 'Purple'], label = ['Leaves', 'Stays'])
plt.xlabel("Total Time With Company(Tenure)")
plt.ylabel("No. Of Customers")
plt.title("Visualization of Relation between Churn and Tech Support")
plt.legend()

In [ ]:
Contract_Churn_Leaves = df1.loc[df1.Churn == 0, 'Contract']
Contract_Churn_Stays  = df1.loc[df1.Churn == 1, 'Contract']

plt.hist([Contract_Churn_Leaves, Contract_Churn_Stays], color = ['Grey', 'Green'], label = ['Leaves', 'Stays'])
plt.xlabel("Total Time With Company(Tenure)")
plt.ylabel("No. Of Customers")
plt.title("Visualization of Relation between Churn and Contract")
plt.legend()

In [ ]:
InternetService_Churn_Leaves = df1.loc[df1.Churn == 0, 'InternetService']
InternetService_Churn_Stays  = df1.loc[df1.Churn == 1, 'InternetService']

plt.hist([InternetService_Churn_Leaves, InternetService_Churn_Stays], color = ['Yellow', 'Pink'], label = ['Leaves', 'Stays'])
plt.xlabel("Total Time With Company(Tenure)")
plt.ylabel("No. Of Customers")
plt.title("Visualization of Relation between Churn and Internet Service")
plt.legend()

In [ ]:
pd.set_option('future.no_silent_downcasting', True)